# Paramterized Ethane Example

In [ ]:
import mbuild as mb 
import mbuild.recipes 
import foyer 
import gmso 

from gmso.external import from_parmed

import warnings 
warnings.filterwarnings('ignore')

## The Ideal MoSDeF Workflow

**mBuild -> Foyer -> GMSO** 

- **mBuild**: Constructing the chemical system 
- **Foyer**: Atomtype and create a parameterized system
- **GMSO**: Store the information related to the chemical system and write out to corresponding file formats.

However, as of right now, integrating **GMSO** into **Foyer** is still WIP in progress. Hence, for demonstration purpose, the current workflow to create a typed `gmso.Topology` is  
**mBuild -> Foyer -> Parmed -> GMOS**

In [ ]:
from mbuild.lib.molecules import Ethane

mb_ethane = Ethane()
oplsaa = foyer.Forcefield(name='oplsaa')

pmd_ethane = oplsaa.apply(mb_ethane)
top = from_parmed(pmd_ethane)
top.box = gmso.Box([10, 10, 10])
top.name = "Ethane"

The `Topology` created will contain all the information about chemical system:
- Atoms each with AtomType
- Connections (bonds, angle, dihedral) each with its `connection_members` and `ConnectionType`

In [ ]:
for site in top.sites:
    print('Atom name, type: {}, {}'.format(site.name, site.atom_type.name))

print()
for bond in top.bonds:
    print('Bond name, connection members: {}\n {}'.format(bond.name, bond.connection_members))

We can take a close look at what are stored in each object:
- **`Atom`** 
- **`Bond`** (other types of **`Connection`** have similar construct) 
- **`AtomType`** (other types of **`Potential`** have similar construct)
- **`PotentialExpression`**

**`Atom`**

In [ ]:
top.sites[0].dict(by_alias=True)

**`Bond`**

In [ ]:
top.bonds[0].dict(by_alias=True)

**`AtomType`**

In [ ]:
top.sites[0].atom_type.dict(by_alias=True)

**`PotentialExpression`**

In [ ]:
potential_object = top.sites[0].atom_type.potential_expression
print('Potential expression: {}'.format(potential_object.expression))
print('Independent variables: {}'.format(potential_object.independent_variables))
print('Paremeters: {}'.format(potential_object.parameters))

## Writers

In [ ]:
from gmso.formats.gro import read_gro, write_gro
from gmso.formats.top import write_top
from gmso.formats.lammpsdata import write_lammpsdata, read_lammpsdata

Once there is a typed `gmso.Topology`, we can write it out to several types of file formats (at least that is our goal). Currently, we can already write out to GROMACS file formats (`.top` and `.gro`) and LAMMPS file format (`.lammps`) 

**GROMACS**

In [ ]:
# Pending new location
write_gro(top, './simulations/gromacs/out.gro')
write_top(top, './simulations/gromacs/out.top')

In [ ]:
top.box

In [ ]:
# Pending new location
# Run a simple energy minimization step
%cd ./simulations/gromacs/
!gmx grompp -v -f em.mdp -p out.top -c out.gro -o em.tpr 
!gmx mdrun -v -deffnm em -s em.tpr
%cd ../../

**LAMMPS**

In [ ]:
# Pending new location 
write_lammpsdata(top, './simulations/lammps/ethane.lammps')

In [ ]:
# Pending new location 
# Run a simple energy minimization step
%cd ./simulations/lammps/
!lmp_mpi -in in.ethane -log minimize.log